# This is a notebook to explore the ligpy results after solving the ODE model

Model results are saved by DDASAC in the format shown in the `ligpy/sample_results/` folder.  Users that rely on a different ODE solver should format their results in the same way in order to use these analysis tools.

In [1]:
# import os
import sys
import fnmatch
import copy
import csv

import numpy as np
import matplotlib.pyplot as plt           
# import cPickle as pickle
import ipywidgets as widgets

import ligpy.ligpy_utils as utils
from ligpy.constants import MW
from ligpy.analysis_tools import load_results


### Load the results from a folder you choose

Users should specify the directory where their results are saved here as `results_dir`.  Then, select the folder you would like to analyze through the rest of this notebook by clicking it in the selection box.

In [5]:
# Absolute or relative path to the directory you've saved model results in
results_dir = 'ligpy/sample_results/'

results = !ls $results_dir
which_result = widgets.Select(options=results)
which_result

In [4]:
(end_time, output_time_step, initial_T, heating_rate, max_T, atol, rtol,
 plant, cool_time, y, t, T, specieslist, speciesindices,
 indices_to_species) = load_results(results_dir + which_result.value)

last_t = t[-1]
last_t_position = len(t)
# convert temperature from K to C
Temperature = T - 273.15

# create a new matrix of mass fractions (instead of concentrations)
m = copy.deepcopy(y)
for species in specieslist:
    # make an array of mass concentration (g/L)
    m[:, speciesindices[species]] = (y[:, speciesindices[species]] *
                                     MW[species][0])
# total mass at time 0
m_0 = m[0,:].sum(axis=0)
# The mass fraction array
m /= m_0

# If you want to make sure all species are present in the MW dictionary run this
# utils.check_species_in_MW(specieslist)

# These two numbers should both be 1 if mass is conserved.  We know that some
# protons are not tracked in the model so this number should be close to 1
print '\nIf mass is conserved %s should equal %s\n'\
        % (m[0,:].sum(axis=0),m[-1,:].sum(axis=0))

There is not a third DDASAC results file (cool down period)

If mass is conserved 1.0 should equal 0.998289709349

